[View in Colaboratory](https://colab.research.google.com/github/amblount/Reinforcement-learning-for-control-systems/blob/alivia-dev/CP_Approach_to_UCSF_Scheduling.ipynb)

In [0]:
!pip install ortools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
col = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
rws = ['111','121', '211', '212', '213', '214','221','231','232','233', '234','241','242','243','244','311','321','331','411','412','413','421','431','441','442','443','451','461','462','463']
df = pd.DataFrame(np.nan, columns = col, index = rws)

In [0]:
df.loc['111', 2] 

nan

In [0]:
counter = 0
for i in col:
   for j in rws:
      counter = counter + 1
      cell = df.loc[j,i]
      print(counter, cell)

1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
24 nan
25 nan
26 nan
27 nan
28 nan
29 nan
30 nan
31 nan
32 nan
33 nan
34 nan
35 nan
36 nan
37 nan
38 nan
39 nan
40 nan
41 nan
42 nan
43 nan
44 nan
45 nan
46 nan
47 nan
48 nan
49 nan
50 nan
51 nan
52 nan
53 nan
54 nan
55 nan
56 nan
57 nan
58 nan
59 nan
60 nan
61 nan
62 nan
63 nan
64 nan
65 nan
66 nan
67 nan
68 nan
69 nan
70 nan
71 nan
72 nan
73 nan
74 nan
75 nan
76 nan
77 nan
78 nan
79 nan
80 nan
81 nan
82 nan
83 nan
84 nan
85 nan
86 nan
87 nan
88 nan
89 nan
90 nan
91 nan
92 nan
93 nan
94 nan
95 nan
96 nan
97 nan
98 nan
99 nan
100 nan
101 nan
102 nan
103 nan
104 nan
105 nan
106 nan
107 nan
108 nan
109 nan
110 nan
111 nan
112 nan
113 nan
114 nan
115 nan
116 nan
117 nan
118 nan
119 nan
120 nan
121 nan
122 nan
123 nan
124 nan
125 nan
126 nan
127 nan
128 nan
129 nan
130 nan
131 nan
132 nan
133 nan
134 nan
135 nan
136 nan
137 nan
138 nan
139 

In [0]:
from __future__ import print_function
import sys
from ortools.constraint_solver import pywrapcp

def main():
# Creates the solver.
  solver = pywrapcp.Solver("ED resident scheduler")
# Creates the variables.
# define variable possibilities
  num_facilities = 4
  num_shifts = 9
  max_res = 4
  residents = 80
  
# define constraint variables which correspond to row names
# example rowName = '111'
# rowName definition
# rowName[0] = '1' facility
# rowName[1] = '1' shift number
# rowName[2] = '1' resident num (some shifts staff multiple residents)
#   facility = solver.IntVar(1, num_facilities, "facility")
#   shifts = solver.IntVar(1, num_shifts, "shifts")
#   res_num = solver.IntVar(1, max_res, "max_res")
  
# define the row names and column names and create the empty dataframe
#   col = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
#   rws = ['111','121', '211', '212', '213', '214','221','231','232','233', '234','241','242','243','244','311','321','331','411','412','413','421','431','441','442','443','451','461','462','463']
#   df = pd.DataFrame(np.nan, columns = col, index = rws)
  
# Create the constraints.
#   label = ''     # Label for displaying results.
  
# define system variables
  num_residents = 80
  num_days = 28
  num_shifts = 30
  
  
# loop through each cell in the empty dataframe 
#   for j in col:
#      for i in rws:
#         cell = df.loc[j,i]
# #       add a resident to a cell
#         residents[(j,i)] = solver.IntVar(1, num_residents, "shift%day%d" % (j,i))
#         shifts[(j, i)] = solver.IntVar(1, num_shifts, "shifts(%i,%i)" % (j, i))
  
#   shifts_flat = [shifts[(j, i)] for j in range(1, num_residents + 1) for i in range(1, num_days + 1)]

  # Create shift variables.
  shifts = {}

  for j in range(1, num_residents + 1):
    for i in range(1, num_days + 1):
      shifts[(j, i)] = solver.IntVar(1, num_shifts + 1, "shifts(%i,%i)" % (j, i))
  shifts_flat = [shifts[(j, i)] for j in range(1, num_residents + 1) for i in range(1, num_days + 1)]

  # Create resident variables.
  residents = {}

  for j in range(1,num_shifts + 1):
    for i in range(1, num_days + 1):
      residents[(j, i)] = solver.IntVar(1, num_residents + 1, "shift%d day%d" % (j,i))

  # Set relationships between shifts and residents.
  for day in range(1, num_days + 1):
    residents_for_day = [residents[(j, day)] for j in range(1, num_shifts + 1)]

    for j in range(1, num_residents + 1):
      s = shifts[(j, day)]
      solver.Add(s.IndexOf(residents_for_day) == j)
      
#  Add system constraints
  
# Each resident works between 20 and 25 days in a week.
  for j in range(num_nurses):
    solver.Add(solver.Sum([shifts[(j, i)] > 0 for i in range(num_days)]) >= 20)
    solver.Add(solver.Sum([shifts[(j, i)] > 0 for i in range(num_days)]) <= 25)
    
# 

        
# Create the decision builder.
  db = solver.Phase(shifts_flat, solver.CHOOSE_FIRST_UNBOUND,
                    solver.ASSIGN_MIN_VALUE)
# Create the solution collector.
  solution = solver.Assignment()
  solution.Add(shifts_flat)
  collector = solver.AllSolutionCollector(solution)
  
  solver.Solve(db, [collector])
  print("Solutions found:", collector.SolutionCount())
  print("Time:", solver.WallTime(), "ms")
  print()
  # Display a few solutions picked at random.
  a_few_solutions = [859, 2034, 5091, 7003]

  for sol in a_few_solutions:
    print("Solution number" , sol, '\n')

    for i in range(1, num_days + 1):
      print("Day", i)
      for j in range(1, num_residents + 1):
        print("resident", j, "assigned to task",
              collector.Value(sol, shifts[(j, i)]))
      print()
  
main()